# 使用 MCP 的工具 ⏰

模型上下文协议（MCP）提供了一种标准化的方式来将 AI 智能体连接到外部工具和数据源。让我们使用 `langchain-mcp-adapters` 连接到 MCP 服务器。

## 设置

加载和/或检查所需的环境变量

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env("example.env")

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# 连接到 mcp-time 服务器以进行时区感知操作
# 这个基于 Go 的服务器提供当前时间、相对时间解析、
# 时区转换、持续时间算术和时间比较等工具
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# 从 MCP 服务器加载工具
mcp_tools = await mcp_client.get_tools()
print(f"已加载 {len(mcp_tools)} 个 MCP 工具：{[t.name for t in mcp_tools]}")

使用 MCP 提供的时间工具创建智能体。

In [ ]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model="openai:gpt-5",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

询问旧金山的当前时间。

In [ ]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()